In [ ]:
#LWAF(Method-1)
import cv2
import numpy as np

def estimate_veiling(image):
    dark_prior = np.min(image, axis=2)
    veiling = 1 - dark_prior / 255  #
    cv2.imshow('Estimated Veiling', (veiling * 255).astype(np.uint8))
    cv2.waitKey(0)
    return veiling
def adaptive_wiener_filter(observation, local_mean, local_variance, noise_variance):
    refined_estimate = local_mean + ((local_variance - noise_variance) / local_variance) * (observation - local_mean)
    return refined_estimate
def calculate_local_variance(image, window_size):
    height, width = image.shape
    pad = window_size // 2
    padded_image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REFLECT)
    local_variance = np.zeros_like(image, dtype=np.float32)
    for y in range(pad, height + pad):
        for x in range(pad, width + pad):
            window = padded_image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            local_mean = np.mean(window)
            local_variance[y - pad, x - pad] = np.mean((window - local_mean) ** 2)
    return local_variance
def estimate_noise_variance(image):
    observation_variances = np.var(image, axis=(0, 1))
    noise_variance = np.mean(observation_variances) / (image.shape[0] * image.shape[1])
    return noise_variance
def defog_image(image, veiling, noise_variance):
    window_size = 32
    local_mean = cv2.blur(veiling, (window_size, window_size))
    local_variance = calculate_local_variance(veiling, window_size)
    refined_veiling = adaptive_wiener_filter(veiling, local_mean, local_variance, noise_variance)
    cv2.imshow('Refined Veiling Estimate', (refined_veiling * 255).astype(np.uint8))
    cv2.waitKey(0)
    transmission = 1 - refined_veiling
    cv2.imshow('Transmission Map', (transmission * 255).astype(np.uint8))
    cv2.waitKey(0)
    defogged_image = (image - transmission[..., None] * 255) / np.maximum(transmission[..., None], 0.01) + transmission[..., None] * 255
    defogged_image = np.clip(defogged_image, 0, 255).astype(np.uint8)
    cv2.imshow('Defogged Image', defogged_image)
    cv2.waitKey(0)
    return defogged_image

foggy_image = cv2.imread('F1.png')
cv2.imshow('Foggy Image', foggy_image)
cv2.waitKey(0)
# Estimate veiling from the foggy image
veiling_estimate = estimate_veiling(foggy_image)
# Decompose the foggy image into low and high-resolution components
low_res_image = cv2.pyrDown(foggy_image)
high_res_image = foggy_image - cv2.pyrUp(low_res_image)
# Estimate noise variance using the low-resolution component
noise_variance = estimate_noise_variance(low_res_image)
# Defog the image with the estimated veiling and noise variance
defogged_image = defog_image(foggy_image, veiling_estimate, noise_variance)

cv2.destroyAllWindows()

In [ ]:
#Guided filter
import cv2
import os
import math
import numpy as np

def DarkChannel(im, sz):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark

def AtmLight(im, dark):
    [h, w] = im.shape[:2]
    imsz = h * w
    numpx = int(max(math.floor(imsz / 1000), 1))
    darkvec = dark.reshape(imsz)
    imvec = im.reshape(imsz, 3)
    indices = darkvec.argsort()
    indices = indices[imsz - numpx::]

    atmsum = np.zeros([1, 3])
    for ind in range(1, numpx):
        atmsum = atmsum + imvec[indices[ind]]
    A = atmsum / numpx
    return A

def TransmissionEstimate(im, A, sz):
    omega = 0.95
    im3 = np.empty(im.shape, im.dtype)
    for ind in range(0, 3):
        im3[:, :, ind] = im[:, :, ind] / A[0, ind]
    transmission = 1 - omega * DarkChannel(im3, sz)
    return transmission

def Guidedfilter(im, p, r, eps):
    mean_I = cv2.boxFilter(im, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(im * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p
    mean_II = cv2.boxFilter(im * im, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I
    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I
    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))
    q = mean_a * im + mean_b
    return q

def TransmissionRefine(im, et):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray, et, r, eps)
    return t

def Recover(im, t, A, tx=0.1):
    res = np.empty(im.shape, im.dtype)
    t = cv2.max(t, tx)

    for ind in range(0, 3):
        res[:, :, ind] = (im[:, :, ind] - A[0, ind]) / t + A[0, ind]
    return res

if __name__ == '__main__':
    dataset_folder = 'Foggy Images'
    output_folder = 'output7'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(dataset_folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            input_image = os.path.join(dataset_folder, filename)
            src = cv2.imread(input_image)
            if src is None:
                print(f"Error: Unable to read the input image '{filename}'.")
                continue
            I = src.astype('float64') / 255
            dark = DarkChannel(I, 15)
            A = AtmLight(I, dark)
            te = TransmissionEstimate(I, A, 15)

            # Down-sample the rough transmission map and the guidance image
            te_downsampled = cv2.resize(te, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
            src_downsampled = cv2.resize(src, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)

            cv2.imshow('Transmission Map (Before Refinement with guided filter )', te_downsampled)
            cv2.waitKey(0)

            t = TransmissionRefine(src_downsampled, te_downsampled)

            # Up-sample the refined transmission map
            t_upsampled = cv2.resize(t, (src.shape[1], src.shape[0]), interpolation=cv2.INTER_LINEAR)

            J = Recover(I, t_upsampled, A, 0.1)

            cv2.imshow('Transmission Map (After Refinement with guided filter)', t_upsampled)
            cv2.waitKey(0)
            cv2.imshow('Input Image', src)
            cv2.waitKey(0)
            cv2.imshow('Output Image', J)
            cv2.waitKey(0)

            # Save the output images
            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, J * 255)  # Convert back to 0-255 range before saving

    cv2.destroyAllWindows()
    print("Folder Saved")


In [ ]:
import os
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def calculate_metrics(input_folder, processed_folder):
    input_images = os.listdir(input_folder)
    processed_images = os.listdir(processed_folder)
    input_images.sort()
    processed_images.sort()
    for input_image_name, processed_image_name in zip(input_images, processed_images):
        input_image_path = os.path.join(input_folder, input_image_name)
        processed_image_path = os.path.join(processed_folder, processed_image_name)
        input_image = cv2.imread(input_image_path)
        processed_image = cv2.imread(processed_image_path)
        input_image = cv2.resize(input_image, (processed_image.shape[1], processed_image.shape[0]))
        input_gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        processed_gray = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)

        # Calculate PSNR
        psnr_value = psnr(input_gray, processed_gray)

        # Calculate SSIM
        ssim_value, _ = ssim(input_gray, processed_gray, full=True)

        print(f"Image: {input_image_name}")
        print("PSNR:", psnr_value)
        print("SSIM:", ssim_value)
        print("="*30)

input_folder = "Foggy Images"
processed_folder = "output7"
calculate_metrics(input_folder, processed_folder)

